<p style="font-size:32px;text-align:center"> <b>Amazon.com - Employee Access Challenge</b> </p>

### Problem statement: 
Predict an employee's access needs, given his/her job role

### Problem Description:
When an employee at any company starts work, they first need to obtain the computer access necessary to fulfill their role. This access may allow an employee to read/manipulate resources through various applications or web portals. It is assumed that employees fulfilling the functions of a given role will access the same or similar resources. It is often the case that employees figure out the access they need as they encounter roadblocks during their daily work (e.g. not able to log into a reporting portal). A knowledgeable supervisor then takes time to manually grant the needed access in order to overcome access obstacles. As employees move throughout a company, this access discovery/recovery cycle wastes a nontrivial amount of time and money.

There is a considerable amount of data regarding an employee’s role within an organization and the resources to which they have access. Given the data related to current employees and their provisioned access, models can be built that automatically determine access privileges as employees enter and leave roles within a company. These auto-access models seek to minimize the human involvement required to grant or revoke employee access.

### Data Overview
Taken data from Amazon.com - Employee Access Challenge on kaggle https://www.kaggle.com/c/amazon-employee-access-challenge
The data consists of real historical data collected from 2010 & 2011.  Employees are manually allowed or denied access to resources over time. You must create an algorithm capable of learning from this historical data to predict approval/denial for an unseen set of employees. 
    - Data columns (total 10 columns):  


<table>
  <col width="130">
  <col width="10">
  <tr>
    <th>Column Name</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>ACTION</td>
    <td>ACTION is 1 if the resource was approved, 0 if the resource was not</td>
  </tr>
  <tr>
    <td>RESOURCE</td>
    <td>An ID for each resource</td>
  </tr>
    <tr>
    <td>MGR_ID</td>
    <td>The EMPLOYEE ID of the manager of the current EMPLOYEE ID record; an employee may have only one manager at a time</td>
  </tr>
    <tr>
    <td>ROLE_ROLLUP_1</td>
    <td>Company role grouping category id 1 (e.g. US Engineering)</td>
  </tr>
    <tr>
    <td>ROLE_ROLLUP_2</td>
    <td>Company role grouping category id 2 (e.g. US Retail)</td>
  </tr>
    <tr>
    <td>ROLE_DEPTNAME</td>
    <td>Company role department description (e.g. Retail)</td>
  </tr>
    <tr>
    <td>ROLE_TITLE</td>
    <td>Company role business title description (e.g. Senior Engineering Retail Manager)</td>
  </tr>
    <tr>
    <td>ROLE_FAMILY_DESC</td>
    <td>Company role family extended description (e.g. Retail Manager, Software Engineering)</td>
  </tr>
    <tr>
    <td>ROLE_FAMILY</td>
    <td>Company role family description (e.g. Retail Manager)
</td>
  </tr>
    <tr>
    <td>ROLE_CODE</td>
    <td>Company role code; this code is unique to each role (e.g. Manager)
</td>
  </tr>
</table>


### Mapping the problem into supervised learning problem:
- Generated new features like Pair wise column featurization. Considered pairs of 2 columns as well as pairs of 3 columns, and analysed if these feature pairs help to improve the model prediction. 

### Business objectives and constraints:  
- The objective of this competition is to build a model, learned using historical data, that will determine an employee's access needs, such that manual access transactions (grants and revokes) are minimized as the employee's attributes change over time. The model will take an employee's role information and a resource code and will return whether or not access should be granted.
- Latency is not an issue here.

### Performance metric for supervised learning:  
- AUC (As mentioned in the competition evaluation criteria)

In [4]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer #Equivalent to CountVectorizer followed by TfidfTransformer.

from sklearn.feature_extraction.text import CountVectorizer ##Convert a collection of text documents to a matrix of token counts
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from tqdm import tqdm
import pickle
import os
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from itertools import combinations
import xgboost as xgb
from sklearn import metrics



In [1]:
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Changing to the required location in the drive
%cd '/content/drive/My Drive/amazon-employee-access-challenge/'

/content/drive/My Drive/amazon-employee-access-challenge


In [5]:
# https://stackoverflow.com/questions/41186565/python-creating-user-input-form-and-converting-inputs-into-data-frame?rq=1

df = pd.DataFrame(columns=['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE'])
parts = int(input("Enter the number of rows:"))

for i in range(parts):
    print('Taking input for row : ',int(i)+1)
    RESOURCE = input("Enter RESOURCE : ")
    MGR_ID = input("Enter MGR_ID : ")
    ROLE_ROLLUP_1 = input("Enter ROLE_ROLLUP_1 : ")
    ROLE_ROLLUP_2 = input("Enter ROLE_ROLLUP_2 : ")
    ROLE_DEPTNAME = input("Enter ROLE_DEPTNAME : ")
    ROLE_TITLE = input("Enter ROLE_TITLE : ")
    ROLE_FAMILY_DESC = input("Enter ROLE_TITLE : ")
    ROLE_FAMILY = input("Enter ROLE_FAMILY : ")
    ROLE_CODE = input("Enter ROLE_CODE : ")

    print("="*20)


    df1 = pd.DataFrame(data=[[RESOURCE, MGR_ID, ROLE_ROLLUP_1, ROLE_ROLLUP_2, ROLE_DEPTNAME, ROLE_TITLE, ROLE_FAMILY_DESC, ROLE_FAMILY, ROLE_CODE]],columns=['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE'])
    df = pd.concat([df,df1], axis=0)

df.index = range(len(df.index))
print(df)

Enter the number of rows:2
Taking input for row :  1
Enter RESOURCE : 39353
Enter MGR_ID : 85475
Enter ROLE_ROLLUP_1 : 117961
Enter ROLE_ROLLUP_2 : 118300
Enter ROLE_DEPTNAME : 123472
Enter ROLE_TITLE : 117905
Enter ROLE_TITLE : 117906
Enter ROLE_FAMILY : 290919
Enter ROLE_CODE : 117908
Taking input for row :  2
Enter RESOURCE : 45333
Enter MGR_ID : 14561
Enter ROLE_ROLLUP_1 : 117951
Enter ROLE_ROLLUP_2 : 117952
Enter ROLE_DEPTNAME : 118008
Enter ROLE_TITLE : 118568
Enter ROLE_TITLE : 118568
Enter ROLE_FAMILY : 19721
Enter ROLE_CODE : 118570
  RESOURCE MGR_ID ROLE_ROLLUP_1  ... ROLE_FAMILY_DESC ROLE_FAMILY ROLE_CODE
0    39353  85475        117961  ...           117906      290919    117908
1    45333  14561        117951  ...           118568       19721    118570

[2 rows x 9 columns]


In [6]:

# https://stackoverflow.com/questions/58020251/how-to-save-classifier-in-sklearn-with-countvectorizer-and-tfidftransformer
def final_fun_1(X):
  df = pd.DataFrame(X, columns = ['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME','ROLE_TITLE','ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE'])
  result_test = df.drop(columns=['ROLE_TITLE'])

  print('Data preprocessing- Pair Wise featurization (Pairs of 2 and Pairs of 3).....')
  # https://stackoverflow.com/questions/43347939/all-possible-combinations-of-columns-in-dataframe-pandas-python
  # this helper function combines columns in pairs of 2. We will find out if this feature engineering technique creates a better ML model
  # the result of pairing is columnX_columnY
  def group_data_2(res):
    df = res
    cc = list(combinations(df.columns,2))
    df = pd.concat([df[c[0]] + '_' + df[c[1]] for c in cc], axis=1, keys=cc)
    df.columns = df.columns.map('_'.join)
    return df

  # this function combines columns in pair of 3
  # the result of pairing is columnX_columnY_columnZ
  def group_data_3(res):
    df = res
    cc = list(combinations(df.columns,3))
    df = pd.concat([df[c[0]] + '_' + df[c[1]] + '_' + df[c[2]] for c in cc], axis=1, keys=cc)
    df.columns = df.columns.map('_'.join)
    return df


  # Pair-wise featurization of Test data
  # feature engineering: combining columns in pairs of 2
  # the result is columnX_columnY and then concatinating all the singular columns with the resultant pairwise combination.
  result2 = group_data_2(result_test)
  result3 = group_data_3(result_test)
  frames = [result_test,result2,result3]
  result_test = pd.concat(frames,axis=1)

  # The total columns should be 8C2 + 8C3 + 8 = 92

  # Storing the column names in a list, we will use this later in SVD and countVectorization
  list_col = list(result_test.columns)

  print("Frequency Encoding the test data......")
  pickle_in = open("freq_frame.pickle","rb")
  data_frame = pickle.load(pickle_in)
  pickle_in.close()

  test_X = result_test.copy()
  new_dataset = pd.DataFrame()
  for col in test_X.columns:
    freq = (test_X[[col]].merge(data_frame[col], on = col, how = 'left'))[col+"_freq"]
    new_dataset[col+"_freq"] = freq
  new_dataset = new_dataset.fillna(0)
  test_freq = new_dataset.copy()

  print('Target Encoding test data....')
  pickle_in = open("data_frame.pickle","rb")
  data_frame = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("global_mean.pickle","rb")
  global_mean = pickle.load(pickle_in)
  pickle_in.close()

  test_X = result_test.copy()
  for c in test_X.columns:
    test_X[c] = (test_X[[c]].merge(data_frame[c], on = c, how = 'left'))["target"]
  test_X = test_X.fillna(global_mean)
  test_target = test_X.copy()

  print('Adding SVD transformed features to test data.....')
  ## Declaring a dictionary for SVD features
  cv= {} # Dictionary for CountVectorizer
  svd = {} # Dictionary for SVD result, n_component = 1
  result_test_svd_tfidf = pd.DataFrame()


  list_test = []
  for name in list_col:
    val2 = 'X_test' + '_' + name 
    list_test.append(val2)

  X_test = result_test.copy()

  from sklearn.decomposition import TruncatedSVD
  from sklearn.feature_extraction.text import TfidfVectorizer

  pickle_in = open("vectorizer.pickle","rb")
  vectorizer = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("dim_red.pickle","rb")
  dim_red = pickle.load(pickle_in)
  pickle_in.close()

  for index,name in enumerate(list_test):
      name = name[7:]
      svd_name = name + '_svd'
      cv[list_test[index]] = vectorizer[index].transform(X_test[name].values)
      # Using Truncated SVD
      n_components=1
      svd[list_test[index]] = dim_red[index].transform(cv[list_test[index]])
      # Storing the SVD in the dataframe
      for i in range(n_components):
          k=str(i)
          result_test_svd_tfidf[svd_name+'_'+ k] = svd[list_test[index]][:,i]

  print('Concatinating target encoded features, frequency encoded features and SVD transformed features....')
  ## Concatinating the dataframes
  result_test_final = pd.concat([test_target, result_test_svd_tfidf, test_freq], axis=1)
  pickle_out = open("df.pickle","wb")
  pickle.dump(result_test_final, pickle_out)
  pickle_out.close()

  X_te = result_test_final.values

  print('Prediction result using the Saved ML model')

  # Load the Model back from file
  Pkl_Filename = "model_new_xgb.pkl"  
  with open(Pkl_Filename, 'rb') as file:  
      model = pickle.load(file)
  
  predictions = model.predict_proba(X_te)[:,1]
  print('The Predicted value from the ML model is : ', predictions)

In [7]:
final_fun_1(df)

Data preprocessing- Pair Wise featurization (Pairs of 2 and Pairs of 3).....
Frequency Encoding the test data......
Target Encoding test data....
Adding SVD transformed features to test data.....
Concatinating target encoded features, frequency encoded features and SVD transformed features....
Prediction result using the Saved ML model
The Predicted value from the ML model is :  [0.9937163  0.26806566]


In [ ]:
# https://stackoverflow.com/questions/58020251/how-to-save-classifier-in-sklearn-with-countvectorizer-and-tfidftransformer
def function_2(X,Y):

  df = pd.DataFrame(X, columns = ['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME','ROLE_TITLE','ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE'])
  result_test = df.drop(columns=['ROLE_TITLE'])

  print('Data preprocessing- Pair Wise featurization (Pairs of 2 and Pairs of 3).....')
  # https://stackoverflow.com/questions/43347939/all-possible-combinations-of-columns-in-dataframe-pandas-python
  # this helper function combines columns in pairs of 2. We will find out if this feature engineering technique creates a better ML model
  # the result of pairing is columnX_columnY
  def group_data_2(res):
    df = res
    cc = list(combinations(df.columns,2))
    df = pd.concat([df[c[0]] + '_' + df[c[1]] for c in cc], axis=1, keys=cc)
    df.columns = df.columns.map('_'.join)
    return df

  # this function combines columns in pair of 3
  # the result of pairing is columnX_columnY_columnZ
  def group_data_3(res):
    df = res
    cc = list(combinations(df.columns,3))
    df = pd.concat([df[c[0]] + '_' + df[c[1]] + '_' + df[c[2]] for c in cc], axis=1, keys=cc)
    df.columns = df.columns.map('_'.join)
    return df


  # Pair-wise featurization of Test data
  # feature engineering: combining columns in pairs of 2
  # the result is columnX_columnY and then concatinating all the singular columns with the resultant pairwise combination.
  result2 = group_data_2(result_test)
  result3 = group_data_3(result_test)
  frames = [result_test,result2,result3]
  result_test = pd.concat(frames,axis=1)

  # The total columns should be 8C2 + 8C3 + 8 = 92

  # Storing the column names in a list, we will use this later in SVD and countVectorization
  list_col = list(result_test.columns)

  print("Frequency Encoding the test data......")
  pickle_in = open("freq_frame.pickle","rb")
  data_frame = pickle.load(pickle_in)
  pickle_in.close()

  test_X = result_test.copy()
  new_dataset = pd.DataFrame()
  for col in test_X.columns:
    freq = (test_X[[col]].merge(data_frame[col], on = col, how = 'left'))[col+"_freq"]
    new_dataset[col+"_freq"] = freq
  new_dataset = new_dataset.fillna(0)
  test_freq = new_dataset.copy()

  print('Target Encoding test data....')
  pickle_in = open("data_frame.pickle","rb")
  data_frame = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("global_mean.pickle","rb")
  global_mean = pickle.load(pickle_in)
  pickle_in.close()

  test_X = result_test.copy()
  for c in test_X.columns:
    test_X[c] = (test_X[[c]].merge(data_frame[c], on = c, how = 'left'))["target"]
  test_X = test_X.fillna(global_mean)
  test_target = test_X.copy()

  print('Adding SVD features to test data.....')
  ## Declaring a dictionary for SVD features
  cv= {} # Dictionary for CountVectorizer
  svd = {} # Dictionary for SVD result, n_component = 1
  result_test_svd_tfidf = pd.DataFrame()


  list_test = []
  for name in list_col:
    val2 = 'X_test' + '_' + name 
    list_test.append(val2)

  X_test = result_test.copy()

  from sklearn.decomposition import TruncatedSVD
  from sklearn.feature_extraction.text import TfidfVectorizer

  pickle_in = open("vectorizer.pickle","rb")
  vectorizer = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("dim_red.pickle","rb")
  dim_red = pickle.load(pickle_in)
  pickle_in.close()

  for index,name in enumerate(list_test):
      name = name[7:]
      svd_name = name + '_svd'
      cv[list_test[index]] = vectorizer[index].transform(X_test[name].values)
      # Using Truncated SVD
      n_components=1
      svd[list_test[index]] = dim_red[index].transform(cv[list_test[index]])
      # Storing the SVD in the dataframe
      for i in range(n_components):
          k=str(i)
          result_test_svd_tfidf[svd_name+'_'+ k] = svd[list_test[index]][:,i]

  print('Concatinating Response coded features with SVD features....')
  ## Concatinating the dataframes
  result_test_final = pd.concat([test_target, result_test_svd_tfidf, test_freq], axis=1)
  pickle_out = open("df.pickle","wb")
  pickle.dump(result_test_final, pickle_out)
  pickle_out.close()

  X_te = result_test_final.values

  print('Prediction result using the Saved ML model')
  # Machine-Learning Model

  # Load the Model back from file
  Pkl_Filename = "model_new_xgb.pkl"  
  with open(Pkl_Filename, 'rb') as file:  
      model = pickle.load(file)
  
  predictions = model.predict_proba(X_te)[:,1]
  print('The Predicted value from the ML model is : ', predictions)

  fpr, tpr, thresholds = metrics.roc_curve(Y, predictions, pos_label=1)
  auc_value = metrics.auc(fpr,tpr)

  return auc_value


In [ ]:
auc_value = function_2(df,[1, 0])
print('auc value is: ', auc_value )

Data preprocessing- Pair Wise featurization (Pairs of 2 and Pairs of 3).....
Frequency Encoding the test data......
Target Encoding test data....
Adding SVD features to test data.....
Concatinating Response coded features with SVD features....
Prediction result using the Saved ML model
The Predicted value from the ML model is :  [0.9937163  0.26806566]
auc value is:  1.0
